In [5]:
import keras

vae_gan_celeba = keras.models.load_model("/Users/leo/Downloads/vae_gan_celeba_128.h5")
vae_gan_celeba_decoder = vae_gan_celeba.layers[-1]
vae_gan_celeba_encoder = keras.Model(vae_gan_celeba.layers[0].input, vae_gan_celeba.layers[-2].output)

vlae_gan_celeba = keras.models.load_model("/Users/leo/Downloads/vlae_gan_128_celeba.h5")
vlae_gan_celeba_decoder = vlae_gan_celeba.layers[-1]
vlae_gan_celeba_encoder = keras.Model(vlae_gan_celeba.layers[0].input, (vlae_gan_celeba.layers[-4].output, vlae_gan_celeba.layers[-3].output, vlae_gan_celeba.layers[-2].output))

In [15]:
from tqdm.notebook import trange, tqdm

from models.VLAE import VLAE
from models.VAE import VAE

vlae_celeba = VLAE(input_dim=(128,128,3), log_dir='',
                         inf0_kernels_strides_featuremaps=[(4, 2, 32), (4, 1, 32)],
                         inf1_kernels_strides_featuremaps=[(4, 2, 64), (4, 1, 64)],
                         ladder0_kernels_strides_featuremaps=[(4, 2, 128), (4, 1, 128)],
                         ladder1_kernels_strides_featuremaps=[(4, 2, 256), (4, 1, 256)],
                         ladder2_kernels_strides_featuremaps=[(4, 2, 256), (4, 2, 512), (4, 1, 512), (4, 1, 512)],
                         gen2_num_units=[256, 512],
                         gen1_num_units=[512, 1024],
                         gen0_kernels_strides_featuremaps=[(4, 1, 1024), (4, 2, 512), (4, 2, 256), (4, 2, 128),
                                                           (4, 2, 64), (4, 1, 3)],
                         kernel_visualization_layer=1, num_samples=5,
                         feature_map_layers=[], inner_activation='ReLU',
                         decay_rate=.0, feature_map_reduction_factor=1,
                         z_dims=[2,2,2], dropout_rate=.0, use_dropout=False,
                         use_batch_norm=True)
vlae_celeba.load_weights('/Users/leo/Downloads/vlae_128_celeba.h5')

vae_celeba = VAE(input_dim=(128,128,3), encoder_conv_filters=[32, 64, 128, 256, 512],
                        encoder_conv_kernel_size=[4, 4, 4, 4, 4], encoder_conv_strides=[2, 2, 2, 2, 1],
                        decoder_conv_t_filters=[512, 256, 128, 3],
                        decoder_conv_t_kernel_size=[4, 4, 4, 4, 4],
                        decoder_conv_t_strides=[2, 2, 2, 2, 1], log_dir='', z_dims=[8],
                        kernel_visualization_layer=1,
                        feature_map_layers=[], use_batch_norm=False,
                        use_dropout=False,
                        decay_rate=.0, num_samples=5,
                        feature_map_reduction_factor=1,
                        inner_activation='ReLU', dropout_rate=.0)

vae_celeba.load_weights('/Users/leo/Downloads/vae_128_celeba.h5')

In [147]:
import os, csv

celeba_attrs = {}
celeba_headers = []

with open(os.path.join('/Users/leo/Downloads/29561_37705_bundle_archive/list_attr_celeba.csv'), newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar="'")
    for i, row in enumerate(spamreader):
        if i == 0:
            print(row)
            for header in row:
                celeba_attrs.setdefault(header, [])
                celeba_headers.append(header)
        else:
            for j, cell in enumerate(row):
                celeba_attrs[celeba_headers[j]].append(cell)

import os
from PIL import Image

from skimage.io import imread



def image_generator(directory, idxs, batch_size = 64):
    files = os.listdir(directory)
    idxs = np.array(idxs)
    files = np.array(celeba_attrs['image_id'])[idxs].flatten()
    global image_ids 
    image_ids = np.array(celeba_attrs['5_o_Clock_Shadow'], dtype=np.float32)[idxs]
    
    num_batch = 0
    while True:
        # Select files (paths/indices) for the batch
        batch_paths  = files[num_batch*batch_size:(num_batch+1)*batch_size]
        batch_input  = []
          
        # Read in each input, perform preprocessing and get labels
        for input_path in batch_paths:
            img = np.array(Image.open(os.path.join(directory, input_path)).resize((128,128))) / 255.0
            
            # input = preprocess_input(image=input)
            batch_input += [ img ]
        # Return a tuple of (input, output) to feed the network
        batch_x = np.array( batch_input )
        num_batch += 1
        
        yield batch_x

['image_id', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive', 'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose', 'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows', 'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace', 'Wearing_Necktie', 'Young']


In [151]:
import numpy as np
np.random.seed(142)

all_ids = np.arange(0, len(celeba_attrs['image_id']))
np.random.shuffle(all_ids)

In [152]:
batch_size = 32

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

steps = 100
marker_size=10

all_generator = image_generator('/Users/leo/Downloads/29561_37705_bundle_archive/img_align_celeba/img_align_celeba',all_ids, batch_size)
vae_preds = vae_celeba.encoder.predict_generator(all_generator, steps = steps)
print("predicted vae")
all_generator = image_generator('/Users/leo/Downloads/29561_37705_bundle_archive/img_align_celeba/img_align_celeba',all_ids, batch_size)
vae_gan_preds = vae_gan_celeba_encoder.predict_generator(all_generator, steps = steps)
print("predicted vae gan")
all_generator = image_generator('/Users/leo/Downloads/29561_37705_bundle_archive/img_align_celeba/img_align_celeba',all_ids, batch_size)
vlae_preds = vlae_celeba.encoder.predict_generator(all_generator, steps = steps)
print("predicted vlae")
all_generator = image_generator('/Users/leo/Downloads/29561_37705_bundle_archive/img_align_celeba/img_align_celeba',all_ids, batch_size)
vlae_gan_preds = vlae_gan_celeba_encoder.predict_generator(all_generator, steps = steps)
print("predicted vlae gan")
num_preds = steps*batch_size
vae_mus_embedded = TSNE().fit_transform(vae_preds)
vae_gan_mus_embedded = TSNE().fit_transform(vae_gan_preds)
vlae_mus_embedded = [TSNE().fit_transform(p) for p in vlae_preds]
vlae_gan_mus_embedded = [TSNE().fit_transform(p) for p in vlae_gan_preds]

for name in celeba_attrs.keys():
    print(name)
    if name == 'image_id':
        continue
    idxs = np.copy(np.array(celeba_attrs[name], dtype=np.float32))
    
    idxs = idxs[all_ids]
    attr_values = np.array(idxs[:num_preds], dtype=np.float32)
    attr_values += 1.0
    attr_values /= 2.0
    
    fig, ax = plt.subplots(figsize=(4, 4))
    off_attribute_idxs = np.where(attr_values==0.0)
    on_attribute_idxs = np.where(attr_values==1.0)
    ax.scatter(vae_mus_embedded[off_attribute_idxs, 0], vae_mus_embedded[off_attribute_idxs, 1], s=marker_size, alpha=0.5, c = 'r', label="Not {}".format(' '.join(name.split('_'))))    
    ax.scatter(vae_mus_embedded[on_attribute_idxs, 0], vae_mus_embedded[on_attribute_idxs, 1], s=marker_size, alpha=0.5, c = 'g', label="{}".format(' '.join(name.split('_'))))        
    ax.legend()
    fig.suptitle('VAE')
    fig.savefig('vae_celeba_{}'.format(name))
    plt.show(fig)
    plt.close(fig)
    
    fig, ax = plt.subplots(figsize=(4, 4))
    off_attribute_idxs = np.where(attr_values==0.0)
    on_attribute_idxs = np.where(attr_values==1.0)
    ax.scatter(vae_gan_mus_embedded[off_attribute_idxs, 0], vae_gan_mus_embedded[off_attribute_idxs, 1], s=marker_size, alpha=0.5, c = 'r', label="Not {}".format(' '.join(name.split('_'))))    
    ax.scatter(vae_gan_mus_embedded[on_attribute_idxs, 0], vae_gan_mus_embedded[on_attribute_idxs, 1], s=marker_size, alpha=0.5, c = 'g', label="{}".format(' '.join(name.split('_'))))        
    ax.legend()
    fig.suptitle('VAE GAN')
    fig.savefig('vae_gan_celeba_{}'.format(name))
    plt.show(fig)
    plt.close(fig)
    
    fig, axs = plt.subplots(ncols=3,nrows=1,figsize=(12,4))
    for dim in range(3):
        axs[dim].scatter(vlae_mus_embedded[dim][off_attribute_idxs, 0], vlae_mus_embedded[dim][off_attribute_idxs, 1], s=marker_size, alpha=0.5, c = 'r', label="Not {}".format(' '.join(name.split('_'))))
        axs[dim].scatter(vlae_mus_embedded[dim][on_attribute_idxs, 0], vlae_mus_embedded[dim][on_attribute_idxs, 1], s=marker_size, alpha=0.5, c = 'g', label="{}".format(' '.join(name.split('_'))))            
        axs[dim].legend()
    fig.suptitle('VLAE') 
    fig.savefig('vlae_celeba_{}'.format(name))
    plt.show(fig)
    plt.close(fig)
    
    fig, axs = plt.subplots(ncols=3,nrows=1,figsize=(12,4))
    for dim in range(3):
        axs[dim].scatter(vlae_gan_mus_embedded[dim][off_attribute_idxs, 0], vlae_gan_mus_embedded[dim][off_attribute_idxs, 1], s=marker_size, alpha=0.5, c = 'r', label="Not {}".format(' '.join(name.split('_'))))
        axs[dim].scatter(vlae_gan_mus_embedded[dim][on_attribute_idxs, 0], vlae_gan_mus_embedded[dim][on_attribute_idxs, 1], s=marker_size, alpha=0.5, c = 'g', label="{}".format(' '.join(name.split('_'))))            
        axs[dim].legend()
    fig.suptitle('VLAE GAN')  
    fig.savefig('vlae_gan_celeba_{}'.format(name))
    plt.show(fig)
    plt.close(fig)

predicted vae
predicted vae gan
predicted vlae
predicted vlae gan
